## Imports and config

In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
import partridge as ptg
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
import datetime

import gtfs_utils as gu

alt.renderers.enable('notebook')
alt.data_transformers.enable('json')

sns.set_style("white")
sns.set_context("talk")
sns.set_palette('Set2', 10)

## Getting the data
Three main options:
1. Tap into the source: We have some functions for working with [MoT's FTP](#FTP).
1. Our archive on [Amazon S3](#S3-archive)
1. [TransitFeeds](#TransitFeeds)' Archive

You can go to our redash interface if you prefer to work with SQL (can't attest to what the state of it is at any given moment).

### FTP

In [3]:
get_ftp_dir()

NameError: name 'get_ftp_dir' is not defined

We have the expected file names in constant variables in `gtfs_utils`

In [ ]:
dir(gtfs_utils)

In [ ]:
LOCAL_GTFS_ZIP_PATH = 'data/gtfs_feeds/2018-09-10.zip'
#LOCAL_TAR

## Creating a `partridge` feed
We have a util function for getting a `partridge` feed object by date.  

In [5]:
feed = gu.get_partridge_feed_by_date(LOCAL_GTFS_ZIP_PATH, datetime.date.today())
type(feed)

partridge.gtfs.feed

In [6]:
LOCAL_TARIFF_PATH = 'data/archive/2018-09-10/Tariff.zip'
zones = gu.get_zones_df(LOCAL_TARIFF_PATH)
zones.head()

,stop_code,zone_name
0,2716,סובב ירושלים
1,2718,סובב ירושלים
2,2720,סובב ירושלים
3,2721,סובב ירושלים
4,2747,סובב ירושלים


In [7]:
r = feed.routes.copy()
r[['route_mkt', 'route_direction', 'route_alternative']] = r['route_desc'].str.split('-', expand=True)
r.head()

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_color,route_mkt,route_direction,route_alternative
3,23672,2,NaN,כרמיאל-כרמיאל<->מרכזית המפרץ - קו החוף-חיפה,14122-1-9,2,NaN,14122,1,9
6,23644,2,NaN,ירושלים מלחה-ירושלים<->בית שמש-בית שמש,15124-2-5,2,NaN,15124,2,5
7,23163,2,NaN,אשדוד עד הלום-אשדוד<->רעננה מערב-הרצליה,18141-2-1,2,NaN,18141,2,1
9,23640,2,NaN,נתניה-נתניה<->בית שמש-בית שמש,21129-1-4,2,NaN,21129,1,4
12,20961,2,NaN,חיפה מרכז-חיפה<->רחובות-רחובות,24034-1-16,2,NaN,24034,1,16


In [9]:
r[r.route_id=='2533']

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_color,route_mkt,route_direction,route_alternative
2445,2533,5,66,כרמלית-תל אביב יפו<->ת. מרכזית פ''ת/רציפים עיר...,15066-2-0,3,NaN,15066,2,0


In [23]:
r.route_mkt.value_counts()

11900    69
12174    13
11331    13
18058    12
15101    11
33055    11
41081    11
24041    10
10274    10
27070    10
32020    10
20054    10
15050    10
12185    10
21045     9
10511     9
10480     9
16011     9
13026     9
10110     8
27096     8
33070     8
12350     8
15013     8
19049     8
30072     8
14102     8
22073     8
21070     8
13161     8
         ..
11483     1
33084     1
17161     1
11150     1
34077     1
22024     1
11278     1
25078     1
15135     1
12004     1
11795     1
33094     1
25091     1
45084     1
52082     1
88002     1
14080     1
13012     1
20015     1
33061     1
11825     1
20038     1
32096     1
25014     1
70015     1
26004     1
10264     1
10038     1
12289     1
22087     1
Name: route_mkt, Length: 2721, dtype: int64

In [33]:
feed.trips[feed.trips.route_id.isin(r[r.route_mkt=='11900'].route_id)].groupby('route_id').size().sort_values(ascending=False)

route_id
2886     4
2888     3
2943     2
19090    2
2925     2
2924     2
2923     2
19091    2
2905     2
2894     2
2843     1
2840     1
2841     1
2842     1
2846     1
2844     1
2845     1
2847     1
2848     1
2849     1
2850     1
2837     1
2827     1
2836     1
2828     1
2852     1
2822     1
2821     1
2820     1
2819     1
        ..
2855     1
2859     1
2926     1
2920     1
2919     1
2916     1
2915     1
2914     1
2913     1
2912     1
2911     1
2910     1
2909     1
2908     1
2906     1
2899     1
2898     1
2896     1
2893     1
2892     1
2891     1
2889     1
2885     1
2882     1
2877     1
2871     1
2863     1
2934     1
2861     1
19089    1
Length: 69, dtype: int64

In [24]:
r[r.route_mkt=='11900']

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_color,route_mkt,route_direction,route_alternative
1972,2813,5,910,עתידים-תל אביב יפו<->חזון אי''ש/הרב יעקב לנדא-...,11900-1-10,3,FF9933,11900,1,10
1973,2818,5,915,סוקולוב/דרך ז'בוטינסקי-בני ברק<->בירנבוים/הרב ...,11900-1-15,3,FF9933,11900,1,15
1974,2820,5,917,נחמיה/עזרא-בני ברק<->בן זכאי/הרב הירש-בני ברק-117,11900-1-17,3,FF9933,11900,1,17
1977,2836,5,931,דרך ז'בוטינסקי/אהרונוביץ-בני ברק<->ביה''ס חורב...,11900-1-31,3,FF9933,11900,1,31
1978,2837,5,932,חזון אי''ש/האדמור מנדבורנא-בני ברק<->ביה''ס אה...,11900-1-32,3,FF9933,11900,1,32
1979,2843,5,938,חזון אי''ש/רבי עקיבא-בני ברק<->ביה''ס אהל צביה...,11900-1-38,3,FF9933,11900,1,38
1980,2845,5,904,הרב אברמסקי/מנצור בן שמעון-בני ברק<->בן זכאי/ה...,11900-1-4,3,FF9933,11900,1,4
1981,2847,5,941,רבי עקיבא/שמואל הנביא-בני ברק<->ביה''ס אהל צבי...,11900-1-41,3,FF9933,11900,1,41
1982,2850,5,944,ירושלים/בן זכאי-בני ברק<->ביה''ח מעייני הישועה...,11900-1-44,3,FF9933,11900,1,44
1983,2852,5,946,הירדן/ פנחס-רמת גן<->מוזיאון לאמנות/דרך אבא הל...,11900-1-46,3,FF9933,11900,1,46


In [25]:
r[r.route_mkt=='12174']

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_color,route_mkt,route_direction,route_alternative
1155,3332,3,174,מסוף רידינג-תל אביב יפו<->ת. מרכזית רשל''צ/הור...,12174-1-ב,3,NaN,12174,1,ב
1156,3347,3,174,מגדלי העיר/שדרות יעקב-ראשון לציון<->רדינג-תל א...,12174-2-ש,3,NaN,12174,2,ש
1308,3335,3,174,מסוף רידינג-תל אביב יפו<->יחזקאל הנביא/נחמיה-ר...,12174-1-כ,3,NaN,12174,1,כ
1309,3343,3,174,ברשבסקי/מנחם כהן-ראשון לציון<->רדינג-תל אביב י...,12174-2-נ,3,NaN,12174,2,נ
1310,11538,3,174,ת. רכבת הראשונים-ראשון לציון<->רדינג-תל אביב י...,12174-2-ז,3,NaN,12174,2,ז
1422,3331,3,174,מסוף רידינג-תל אביב יפו<->ת. רכבת הראשונים-ראש...,12174-1-א,3,NaN,12174,1,א
1423,3336,3,174,ת. רכבת הראשונים-ראשון לציון<->רדינג-תל אביב י...,12174-2-#,3,NaN,12174,2,#
1424,3341,3,174,ת. מרכזית רחובות/רציפים-רחובות<->רדינג-תל אביב...,12174-2-ו,3,NaN,12174,2,ו
1425,3345,3,174,מרכז ספורט/שדרות בן גוריון-ראשון לציון<->רדינג...,12174-2-פ,3,NaN,12174,2,פ
6392,3330,3,174,מסוף רידינג-תל אביב יפו<->ת. רכבת הראשונים-ראש...,12174-1-#,3,NaN,12174,1,#


In [26]:
r.groupby('route_mkt').route_short_name.nunique().sort_values(ascending=False)

route_mkt
11900    45
44005     3
50014     2
10254     2
13007     2
13004     2
33028     2
52004     2
13450     2
23013     2
21006     2
29051     2
38091     2
15013     2
18007     2
50012     2
11084     2
21003     2
50008     2
50006     2
33004     2
33002     2
39006     2
49010     2
25002     2
13011     2
18006     2
39012     2
13026     2
34007     2
         ..
44085     0
23042     0
23038     0
14146     0
33024     0
27035     0
32020     0
21139     0
21129     0
46021     0
27060     0
24041     0
14122     0
34047     0
29026     0
49084     0
13118     0
36022     0
31030     0
22142     0
43010     0
26050     0
34090     0
18141     0
38096     0
15124     0
15123     0
33055     0
30072     0
24034     0
Name: route_short_name, Length: 2721, dtype: int64

In [27]:
r[r.route_mkt=='44005']

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_color,route_mkt,route_direction,route_alternative
5329,9495,25,5,החרש/עמל-פתח תקווה<->עבד אל כרים קאסם/אל ח'ליל...,44005-1-2,3,NaN,44005,1,2
5574,9494,25,5א,החרש/עמל-פתח תקווה<->עבד אל כרים קאסם/אל ח'ליל...,44005-1-1,3,NaN,44005,1,1
5575,9498,25,5,עבד אל כרים קאסם/אל ח'ליל-כפר קאסם<->עמל/שלמה ...,44005-2-2,3,NaN,44005,2,2
6002,9493,25,5,החרש/עמל-פתח תקווה<->עבד אל כרים קאסם/אל ח'ליל...,44005-1-0,3,NaN,44005,1,0
6003,9497,25,5 א,עבד אל כרים קאסם/אל ח'ליל-כפר קאסם<->עמל/שלמה ...,44005-2-1,3,NaN,44005,2,1
7828,9496,25,5,עבד אל כרים קאסם/אל ח'ליל-כפר קאסם<->עמל/שלמה ...,44005-2-0,3,NaN,44005,2,0


## Tidy DataFrame
A (monstrous) merged DataFrame for fancy analysis can be got using `get_tidy_feed_df()`, whom you pass a partridge feed and extra dataframes you want to merge to it (only `zones` is used here).

This takes a few minutes (MoT's GTFS is big)

In [14]:
f = gu.get_tidy_feed_df(feed, [zones])

ValueError: invalid timedelta unit s provided

and what you get is this:

In [ ]:
f.head()

In the future I intend to make this more customizable (field selection, transformations and more). 

In [27]:
f.shape

(3151606, 16)

In [28]:
feed.stop_times.shape

(3151606, 8)

So we truly have all the stop times for one whole day of trips.

In [22]:
r = pd.read_pickle('data/gtfs_stats_route_desc/2018-01-01_route_stats.pkl.gz', compression='gzip')
r.head()

,route_id,route_short_name,agency_id,agency_name,route_long_name,route_type,route_mkt,route_direction,route_alternative,num_trips,...,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon,num_stops,start_zone,end_zone,num_zones,num_zones_missing,date
0,1,1,25,אפיקים,תחנת רכבת יבנה מערב-יבנה<->תחנת רכבת מזרח-יבנה-1#,3,67001,1,#,26,...,31.890699,34.731192,31.862016,34.744080,18,רחובות,רחובות,1,0,2018-01-01
1,10,4,25,אפיקים,תחנת רכבת מזרח-יבנה<->תחנת רכבת יבנה מערב-יבנה-2#,3,47004,2,#,35,...,31.862016,34.744080,31.890699,34.731192,25,רחובות,רחובות,1,0,2018-01-01
2,10000,91,14,נתיב אקספרס,קופת חולים/הנשיא-צפת<->דרך חטיבת יפתח-צפת-11,3,27091,1,1,1,...,32.960472,35.495491,32.968445,35.494423,33,גליל עליון ורמת הגולן,גליל עליון ורמת הגולן,1,0,2018-01-01
3,10001,91,14,נתיב אקספרס,האר''י/ישראל ב''ק-צפת<->שפרינצק/קרן היסוד-צפת-2#,3,27091,2,#,5,...,32.971863,35.493324,32.958485,35.494245,26,גליל עליון ורמת הגולן,גליל עליון ורמת הגולן,1,0,2018-01-01
4,10002,92,14,נתיב אקספרס,געתון/מירון-כפר ורדים<->בית ספר כפר ורדים-כפר ...,3,25092,1,#,1,...,32.992835,35.258030,32.996105,35.277132,17,נהריה,נהריה,1,0,2018-01-01


In [23]:
cols = ['agency_name', 'route_mkt', 'route_short_name',
        'cluster_name', 'from_date', 'to_date', 'cluster_id',
        'line_type', 'line_type_desc', 'cluster_sub_desc', 'EXTRA']
ctl = (pd.read_csv('data/archive/2018-07-10/ClusterToLine.zip', encoding='windows-1255',
                   skiprows=[0], header=None, names=cols)
       .drop(columns=['EXTRA']))
ctl.head()

,agency_name,route_mkt,route_short_name,cluster_name,from_date,to_date,cluster_id,line_type,line_type_desc,cluster_sub_desc
0,אגד,10003,3,חיפה עירוני,01/01/2012,01/01/2200,81,1,עירוני,NaN
1,אגד,10005,5,חיפה עירוני,01/01/2012,01/01/2200,81,1,עירוני,NaN
2,אגד,10006,6,חיפה עירוני,01/01/2012,01/01/2200,81,1,עירוני,NaN
3,אגד,10007,7,חיפה עירוני,01/01/2012,01/01/2200,81,1,עירוני,NaN
4,אגד,10008,8,חיפה עירוני,01/01/2012,01/01/2200,81,1,עירוני,NaN


In [24]:
ctl[['route_mkt', 'line_type_desc']].dtypes

route_mkt          int64
line_type_desc    object
dtype: object

In [26]:
r = r.merge(ctl[['route_mkt', 'line_type_desc']].astype(str), how='left',)
r.line_type_desc.value_counts()

עירוני       2739
בינעירוני    2169
אזורי         770
Name: line_type_desc, dtype: int64

In [12]:
from glob import glob
import os
out_folder = 'data\gtfs_stats_csv'
#os.mkdir(out_folder)
for file in glob('data\gtfs_stats_route_desc\*route_stats*'):
    base = os.path.basename(file).split('.')[0]
    out_path = os.path.join(out_folder, base+'.csv')
    if not os.path.exists(out_path):
        print (base)
        r = pd.read_pickle(file, compression='gzip')
        print (r.shape)
        r.to_csv(out_path)


2018-11-20_route_stats
(6651, 43)


In [6]:
out_folder = 'data\siri_csv'
#os.mkdir(out_folder)
for file in glob('data\\siri\\2018-11\\*.parq'):
    print (file)
    base = '.'.join(os.path.basename(file).split('.')[:-2])
    out_path = os.path.join(out_folder, base+'.csv.gz')
    if not os.path.exists(out_path):
        print (base)
        r = pd.read_parquet(file)
        print (r.shape)
        td = pd.to_timedelta(r.time_recorded, unit='s').abs().dt.components.astype('str')
        r['time_recorded'] = td.hours.str.zfill(2) + ':' + td.minutes.str.zfill(2) + ':' + td.seconds.str.zfill(2)
        td = pd.to_timedelta(r.start_time, unit='s').abs().dt.components.astype('str')
        r['start_time'] = td.hours.str.zfill(2) + ':' + td.minutes.str.zfill(2) + ':' + td.seconds.str.zfill(2)
        td = pd.to_timedelta(r.end_time, unit='s').abs().dt.components.astype('str')
        r['end_time'] = td.hours.str.zfill(2) + ':' + td.minutes.str.zfill(2) + ':' + td.seconds.str.zfill(2)
        r = r[['date', 'time_recorded', 'agency', 'bus_id', 'line_num', 'route_id', 'service_id', 'start_time', 'end_time', 
               'lat', 'lon']]
        r = r.rename(columns={'line_num': 'route_short_name', 'agency': 'agency_id'})
        r.to_csv(out_path, compression='gzip', index=False)


data\siri\2018-11\siri_rt_data.2018-11-01.0.log.parq
data\siri\2018-11\siri_rt_data.2018-11-01.1.log.parq
data\siri\2018-11\siri_rt_data.2018-11-01.10.log.parq
data\siri\2018-11\siri_rt_data.2018-11-01.11.log.parq
data\siri\2018-11\siri_rt_data.2018-11-01.12.log.parq
data\siri\2018-11\siri_rt_data.2018-11-01.13.log.parq
data\siri\2018-11\siri_rt_data.2018-11-01.2.log.parq
data\siri\2018-11\siri_rt_data.2018-11-01.3.log.parq
data\siri\2018-11\siri_rt_data.2018-11-01.4.log.parq
data\siri\2018-11\siri_rt_data.2018-11-01.5.log.parq
data\siri\2018-11\siri_rt_data.2018-11-01.6.log.parq
data\siri\2018-11\siri_rt_data.2018-11-01.7.log.parq
data\siri\2018-11\siri_rt_data.2018-11-01.8.log.parq
data\siri\2018-11\siri_rt_data.2018-11-01.9.log.parq
data\siri\2018-11\siri_rt_data.2018-11-03.0.log.parq
siri_rt_data.2018-11-03.0
(805878, 11)
data\siri\2018-11\siri_rt_data.2018-11-03.1.log.parq
siri_rt_data.2018-11-03.1
(88361, 11)
data\siri\2018-11\siri_rt_data.2018-11-04.0.log.parq
siri_rt_data.2018-

In [ ]:
out_folder = 'data\\siri_csv'
for file in glob(out_folder+'\\*'):
    r = pd.read_csv(file)
    r = r.rename(columns={'route_mkt': 'route_id', 'line_num': 'route_short_name', 'agency': 'agency_id'})
    r.to_csv(file, compression='gzip', index=False)
    

In [102]:
path = 'data\\siri\\2018-10\\siri_rt_data.2018-10-08.10.log.parq'
r = pd.read_parquet(path)
r.head()

,agency,bus_id,end_time,line_num,route_id,service_id,start_time,time_recorded,lat,lon,date
0,3,8716501,69060,88,9857,34880487,65400,NaN,32.038010,34.750378,2018-10-08
1,3,7726069,69720,88,9857,34880488,66600,NaN,32.023415,34.760494,2018-10-08
2,3,7726269,71040,88,9857,34880489,67800,NaN,32.008469,34.791424,2018-10-08
3,3,7578969,72120,88,9857,34880490,69000,NaN,0.000000,0.000000,2018-10-08
4,3,7566669,73380,88,9857,34880491,70200,NaN,0.000000,0.000000,2018-10-08


In [ ]:
path = 'data\\siri\\2018-10\\siri_rt_data.2018-10-08.10.log.parq'
td = pd.to_timedelta(r.time_recorded, unit='s').abs().dt.components.astype('str')
r['time_recorded'] = td.hours.str.zfill(2) + ':' + td.minutes.str.zfill(2) + ':' + td.seconds.str.zfill(2)
td = pd.to_timedelta(r.start_time, unit='s').abs().dt.components.astype('str')
r['start_time'] = td.hours.str.zfill(2) + ':' + td.minutes.str.zfill(2) + ':' + td.seconds.str.zfill(2)
td = pd.to_timedelta(r.end_time, unit='s').abs().dt.components.astype('str')
r['end_time'] = td.hours.str.zfill(2) + ':' + td.minutes.str.zfill(2) + ':' + td.seconds.str.zfill(2)
r = r[['date', 'time_recorded', 'agency', 'bus_id', 'line_num', 'route_id', 'service_id', 'start_time', 'end_time', 
       'lat', 'lon']]
r = r.rename(columns={'line_num': 'route_short_name', 'agency': 'agency_id'})
r.head()

In [98]:
r.head()

,date,time_recorded,agency_id,bus_id,route_short_name,route_id,service_id,start_time,end_time,lat,lon
0,2018-10-01,23:47:08,16,6780432,190,14791,30398274,00:30:00,02:19:00,0.000000,0.000000
1,2018-10-01,23:47:08,16,6780432,190,14791,30398274,00:30:00,02:19:00,0.000000,0.000000
2,2018-10-01,00:00:36,7,682B,340,17178,30232152,23:30:00,01:27:00,32.707657,35.157524
3,2018-10-01,21:52:09,3,7766769,11,11387,12418452,00:00:00,00:06:00,0.000000,0.000000
4,2018-10-01,00:00:35,14,426A,471,12159,23694799,22:45:00,00:11:00,32.847446,35.091839


In [ ]:
x = x[['date', 'time_recorded', 'agency', 'bus_id', 'line_num', 'route_id', 'service_id', 'start_time', 'end_time', 
       'lat', 'lon']]

In [88]:
!aws s3 cp s3://s3.obus.hasadna.org.il/2018-10-10.zip data/gtfs_feeds/2018-10-10.zip
!aws s3 cp s3://s3.obus.hasadna.org.il/2018-10-11.zip data/gtfs_feeds/2018-10-11.zip
!aws s3 cp s3://s3.obus.hasadna.org.il/2018-10-12.zip data/gtfs_feeds/2018-10-12.zip
!aws s3 cp s3://s3.obus.hasadna.org.il/2018-10-13.zip data/gtfs_feeds/2018-10-13.zip
!aws s3 cp s3://s3.obus.hasadna.org.il/2018-10-14.zip data/gtfs_feeds/2018-10-14.zip
!aws s3 cp s3://s3.obus.hasadna.org.il/2018-10-15.zip data/gtfs_feeds/2018-10-15.zip
!aws s3 cp s3://s3.obus.hasadna.org.il/2018-10-16.zip data/gtfs_feeds/2018-10-16.zip
!aws s3 cp s3://s3.obus.hasadna.org.il/2018-10-17.zip data/gtfs_feeds/2018-10-17.zip
!aws s3 cp s3://s3.obus.hasadna.org.il/2018-10-18.zip data/gtfs_feeds/2018-10-18.zip
!aws s3 cp s3://s3.obus.hasadna.org.il/2018-10-19.zip data/gtfs_feeds/2018-10-19.zip
!aws s3 cp s3://s3.obus.hasadna.org.il/2018-10-20.zip data/gtfs_feeds/2018-10-20.zip
!aws s3 cp s3://s3.obus.hasadna.org.il/2018-10-21.zip data/gtfs_feeds/2018-10-21.zip
!aws s3 cp s3://s3.obus.hasadna.org.il/2018-10-22.zip data/gtfs_feeds/2018-10-22.zip
!aws s3 cp s3://s3.obus.hasadna.org.il/2018-10-23.zip data/gtfs_feeds/2018-10-23.zip
!aws s3 cp s3://s3.obus.hasadna.org.il/2018-10-24.zip data/gtfs_feeds/2018-10-24.zip
!aws s3 cp s3://s3.obus.hasadna.org.il/2018-10-25.zip data/gtfs_feeds/2018-10-25.zip
!aws s3 cp s3://s3.obus.hasadna.org.il/2018-10-26.zip data/gtfs_feeds/2018-10-26.zip
!aws s3 cp s3://s3.obus.hasadna.org.il/2018-10-27.zip data/gtfs_feeds/2018-10-27.zip
!aws s3 cp s3://s3.obus.hasadna.org.il/2018-10-28.zip data/gtfs_feeds/2018-10-28.zip
!aws s3 cp s3://s3.obus.hasadna.org.il/2018-10-29.zip data/gtfs_feeds/2018-10-29.zip
!aws s3 cp s3://s3.obus.hasadna.org.il/2018-10-30.zip data/gtfs_feeds/2018-10-30.zip
!aws s3 cp s3://s3.obus.hasadna.org.il/2018-10-31.zip data/gtfs_feeds/2018-10-31.zip

Completed 256.0 KiB/140.2 MiB (106.1 KiB/s) with 1 file(s) remaining
Completed 512.0 KiB/140.2 MiB (202.1 KiB/s) with 1 file(s) remaining
Completed 768.0 KiB/140.2 MiB (265.9 KiB/s) with 1 file(s) remaining
Completed 1.0 MiB/140.2 MiB (353.3 KiB/s) with 1 file(s) remaining  
Completed 1.2 MiB/140.2 MiB (440.8 KiB/s) with 1 file(s) remaining  
Completed 1.5 MiB/140.2 MiB (512.0 KiB/s) with 1 file(s) remaining  
Completed 1.8 MiB/140.2 MiB (593.2 KiB/s) with 1 file(s) remaining  
Completed 2.0 MiB/140.2 MiB (658.1 KiB/s) with 1 file(s) remaining  
Completed 2.2 MiB/140.2 MiB (685.1 KiB/s) with 1 file(s) remaining  
Completed 2.5 MiB/140.2 MiB (747.7 KiB/s) with 1 file(s) remaining  
Completed 2.8 MiB/140.2 MiB (777.0 KiB/s) with 1 file(s) remaining  
Completed 3.0 MiB/140.2 MiB (704.7 KiB/s) with 1 file(s) remaining  
Completed 3.2 MiB/140.2 MiB (737.6 KiB/s) with 1 file(s) remaining  
Completed 3.5 MiB/140.2 MiB (776.3 KiB/s) with 1 file(s) remaining  
Completed 3.8 MiB/140.2 MiB (728.9

Completed 256.0 KiB/139.2 MiB (91.9 KiB/s) with 1 file(s) remaining
Completed 512.0 KiB/139.2 MiB (183.3 KiB/s) with 1 file(s) remaining
Completed 768.0 KiB/139.2 MiB (271.9 KiB/s) with 1 file(s) remaining
Completed 1.0 MiB/139.2 MiB (315.7 KiB/s) with 1 file(s) remaining  
Completed 1.2 MiB/139.2 MiB (378.5 KiB/s) with 1 file(s) remaining  
Completed 1.5 MiB/139.2 MiB (448.1 KiB/s) with 1 file(s) remaining  
Completed 1.8 MiB/139.2 MiB (514.8 KiB/s) with 1 file(s) remaining  
Completed 2.0 MiB/139.2 MiB (576.9 KiB/s) with 1 file(s) remaining  
Completed 2.2 MiB/139.2 MiB (644.3 KiB/s) with 1 file(s) remaining  
Completed 2.5 MiB/139.2 MiB (708.9 KiB/s) with 1 file(s) remaining  
Completed 2.8 MiB/139.2 MiB (769.8 KiB/s) with 1 file(s) remaining  
Completed 3.0 MiB/139.2 MiB (837.5 KiB/s) with 1 file(s) remaining  
Completed 3.2 MiB/139.2 MiB (785.1 KiB/s) with 1 file(s) remaining  
Completed 3.5 MiB/139.2 MiB (814.5 KiB/s) with 1 file(s) remaining  
Completed 3.8 MiB/139.2 MiB (840.6 

Completed 256.0 KiB/138.9 MiB (98.8 KiB/s) with 1 file(s) remaining
Completed 512.0 KiB/138.9 MiB (176.4 KiB/s) with 1 file(s) remaining
Completed 768.0 KiB/138.9 MiB (263.1 KiB/s) with 1 file(s) remaining
Completed 1.0 MiB/138.9 MiB (348.5 KiB/s) with 1 file(s) remaining  
Completed 1.2 MiB/138.9 MiB (434.9 KiB/s) with 1 file(s) remaining  
Completed 1.5 MiB/138.9 MiB (517.9 KiB/s) with 1 file(s) remaining  
Completed 1.8 MiB/138.9 MiB (583.1 KiB/s) with 1 file(s) remaining  
Completed 2.0 MiB/138.9 MiB (660.4 KiB/s) with 1 file(s) remaining  
Completed 2.2 MiB/138.9 MiB (738.0 KiB/s) with 1 file(s) remaining  
Completed 2.5 MiB/138.9 MiB (773.4 KiB/s) with 1 file(s) remaining  
Completed 2.8 MiB/138.9 MiB (738.5 KiB/s) with 1 file(s) remaining  
Completed 3.0 MiB/138.9 MiB (794.6 KiB/s) with 1 file(s) remaining  
Completed 3.2 MiB/138.9 MiB (843.2 KiB/s) with 1 file(s) remaining  
Completed 3.5 MiB/138.9 MiB (902.3 KiB/s) with 1 file(s) remaining  
Completed 3.8 MiB/138.9 MiB (955.2 

Completed 256.0 KiB/134.5 MiB (90.7 KiB/s) with 1 file(s) remaining
Completed 512.0 KiB/134.5 MiB (176.9 KiB/s) with 1 file(s) remaining
Completed 768.0 KiB/134.5 MiB (263.5 KiB/s) with 1 file(s) remaining
Completed 1.0 MiB/134.5 MiB (345.6 KiB/s) with 1 file(s) remaining  
Completed 1.2 MiB/134.5 MiB (425.1 KiB/s) with 1 file(s) remaining  
Completed 1.5 MiB/134.5 MiB (508.3 KiB/s) with 1 file(s) remaining  
Completed 1.8 MiB/134.5 MiB (581.3 KiB/s) with 1 file(s) remaining  
Completed 2.0 MiB/134.5 MiB (644.6 KiB/s) with 1 file(s) remaining  
Completed 2.2 MiB/134.5 MiB (710.9 KiB/s) with 1 file(s) remaining  
Completed 2.5 MiB/134.5 MiB (698.1 KiB/s) with 1 file(s) remaining  
Completed 2.8 MiB/134.5 MiB (741.8 KiB/s) with 1 file(s) remaining  
Completed 3.0 MiB/134.5 MiB (770.5 KiB/s) with 1 file(s) remaining  
Completed 3.2 MiB/134.5 MiB (810.9 KiB/s) with 1 file(s) remaining  
Completed 3.5 MiB/134.5 MiB (849.1 KiB/s) with 1 file(s) remaining  
Completed 3.8 MiB/134.5 MiB (908.0 

Completed 256.0 KiB/130.7 MiB (111.9 KiB/s) with 1 file(s) remaining
Completed 512.0 KiB/130.7 MiB (216.0 KiB/s) with 1 file(s) remaining
Completed 768.0 KiB/130.7 MiB (316.6 KiB/s) with 1 file(s) remaining
Completed 1.0 MiB/130.7 MiB (403.0 KiB/s) with 1 file(s) remaining  
Completed 1.2 MiB/130.7 MiB (410.1 KiB/s) with 1 file(s) remaining  
Completed 1.5 MiB/130.7 MiB (488.9 KiB/s) with 1 file(s) remaining  
Completed 1.8 MiB/130.7 MiB (546.3 KiB/s) with 1 file(s) remaining  
Completed 2.0 MiB/130.7 MiB (624.2 KiB/s) with 1 file(s) remaining  
Completed 2.2 MiB/130.7 MiB (697.3 KiB/s) with 1 file(s) remaining  
Completed 2.5 MiB/130.7 MiB (770.6 KiB/s) with 1 file(s) remaining  
Completed 2.8 MiB/130.7 MiB (838.8 KiB/s) with 1 file(s) remaining  
Completed 3.0 MiB/130.7 MiB (870.3 KiB/s) with 1 file(s) remaining  
Completed 3.2 MiB/130.7 MiB (934.0 KiB/s) with 1 file(s) remaining  
Completed 3.5 MiB/130.7 MiB (911.0 KiB/s) with 1 file(s) remaining  
Completed 3.8 MiB/130.7 MiB (858.3

...

Completed 256.0 KiB/183.2 MiB (42.0 KiB/s) with 1 file(s) remaining
Completed 512.0 KiB/183.2 MiB (80.6 KiB/s) with 1 file(s) remaining
Completed 768.0 KiB/183.2 MiB (120.6 KiB/s) with 1 file(s) remaining
Completed 1.0 MiB/183.2 MiB (140.9 KiB/s) with 1 file(s) remaining  
Completed 1.2 MiB/183.2 MiB (156.1 KiB/s) with 1 file(s) remaining  
Completed 1.5 MiB/183.2 MiB (178.8 KiB/s) with 1 file(s) remaining  
Completed 1.8 MiB/183.2 MiB (198.3 KiB/s) with 1 file(s) remaining  
Completed 2.0 MiB/183.2 MiB (214.9 KiB/s) with 1 file(s) remaining  
Completed 2.2 MiB/183.2 MiB (222.9 KiB/s) with 1 file(s) remaining  
Completed 2.5 MiB/183.2 MiB (241.9 KiB/s) with 1 file(s) remaining  
Completed 2.8 MiB/183.2 MiB (263.7 KiB/s) with 1 file(s) remaining  
Completed 3.0 MiB/183.2 MiB (279.9 KiB/s) with 1 file(s) remaining  
Completed 3.2 MiB/183.2 MiB (297.0 KiB/s) with 1 file(s) remaining  
Completed 3.5 MiB/183.2 MiB (317.3 KiB/s) with 1 file(s) remaining  
Completed 3.8 MiB/183.2 MiB (339.9 K

Completed 256.0 KiB/169.1 MiB (100.9 KiB/s) with 1 file(s) remaining
Completed 512.0 KiB/169.1 MiB (180.9 KiB/s) with 1 file(s) remaining
Completed 768.0 KiB/169.1 MiB (263.9 KiB/s) with 1 file(s) remaining
Completed 1.0 MiB/169.1 MiB (344.7 KiB/s) with 1 file(s) remaining  
Completed 1.2 MiB/169.1 MiB (417.6 KiB/s) with 1 file(s) remaining  
Completed 1.5 MiB/169.1 MiB (484.2 KiB/s) with 1 file(s) remaining  
Completed 1.8 MiB/169.1 MiB (529.6 KiB/s) with 1 file(s) remaining  
Completed 2.0 MiB/169.1 MiB (583.8 KiB/s) with 1 file(s) remaining  
Completed 2.2 MiB/169.1 MiB (644.5 KiB/s) with 1 file(s) remaining  
Completed 2.5 MiB/169.1 MiB (712.5 KiB/s) with 1 file(s) remaining  
Completed 2.8 MiB/169.1 MiB (774.3 KiB/s) with 1 file(s) remaining  
Completed 3.0 MiB/169.1 MiB (837.3 KiB/s) with 1 file(s) remaining  
Completed 3.2 MiB/169.1 MiB (880.0 KiB/s) with 1 file(s) remaining  
Completed 3.5 MiB/169.1 MiB (844.7 KiB/s) with 1 file(s) remaining  
Completed 3.8 MiB/169.1 MiB (853.7

Completed 256.0 KiB/179.4 MiB (33.9 KiB/s) with 1 file(s) remaining
Completed 512.0 KiB/179.4 MiB (66.8 KiB/s) with 1 file(s) remaining
Completed 768.0 KiB/179.4 MiB (87.7 KiB/s) with 1 file(s) remaining
Completed 1.0 MiB/179.4 MiB (112.9 KiB/s) with 1 file(s) remaining 
Completed 1.2 MiB/179.4 MiB (121.3 KiB/s) with 1 file(s) remaining 
Completed 1.5 MiB/179.4 MiB (144.6 KiB/s) with 1 file(s) remaining 
Completed 1.8 MiB/179.4 MiB (167.8 KiB/s) with 1 file(s) remaining 
Completed 2.0 MiB/179.4 MiB (166.2 KiB/s) with 1 file(s) remaining 
Completed 2.2 MiB/179.4 MiB (180.8 KiB/s) with 1 file(s) remaining 
Completed 2.5 MiB/179.4 MiB (195.7 KiB/s) with 1 file(s) remaining 
Completed 2.8 MiB/179.4 MiB (213.8 KiB/s) with 1 file(s) remaining 
Completed 3.0 MiB/179.4 MiB (226.7 KiB/s) with 1 file(s) remaining 
Completed 3.2 MiB/179.4 MiB (244.8 KiB/s) with 1 file(s) remaining 
Completed 3.5 MiB/179.4 MiB (260.4 KiB/s) with 1 file(s) remaining 
Completed 3.8 MiB/179.4 MiB (266.4 KiB/s) with 1

Completed 256.0 KiB/174.0 MiB (86.7 KiB/s) with 1 file(s) remaining
Completed 512.0 KiB/174.0 MiB (172.4 KiB/s) with 1 file(s) remaining
Completed 768.0 KiB/174.0 MiB (255.7 KiB/s) with 1 file(s) remaining
Completed 1.0 MiB/174.0 MiB (329.3 KiB/s) with 1 file(s) remaining  
Completed 1.2 MiB/174.0 MiB (410.3 KiB/s) with 1 file(s) remaining  
Completed 1.5 MiB/174.0 MiB (474.2 KiB/s) with 1 file(s) remaining  
Completed 1.8 MiB/174.0 MiB (527.1 KiB/s) with 1 file(s) remaining  
Completed 2.0 MiB/174.0 MiB (601.8 KiB/s) with 1 file(s) remaining  
Completed 2.2 MiB/174.0 MiB (666.9 KiB/s) with 1 file(s) remaining  
Completed 2.5 MiB/174.0 MiB (714.1 KiB/s) with 1 file(s) remaining  
Completed 2.8 MiB/174.0 MiB (682.2 KiB/s) with 1 file(s) remaining  
Completed 3.0 MiB/174.0 MiB (716.4 KiB/s) with 1 file(s) remaining  
Completed 3.2 MiB/174.0 MiB (754.3 KiB/s) with 1 file(s) remaining  
Completed 3.5 MiB/174.0 MiB (810.7 KiB/s) with 1 file(s) remaining  
Completed 3.8 MiB/174.0 MiB (854.7 

Completed 256.0 KiB/174.0 MiB (80.6 KiB/s) with 1 file(s) remaining
Completed 512.0 KiB/174.0 MiB (161.0 KiB/s) with 1 file(s) remaining
Completed 768.0 KiB/174.0 MiB (234.8 KiB/s) with 1 file(s) remaining
Completed 1.0 MiB/174.0 MiB (310.2 KiB/s) with 1 file(s) remaining  
Completed 1.2 MiB/174.0 MiB (382.8 KiB/s) with 1 file(s) remaining  
Completed 1.5 MiB/174.0 MiB (453.1 KiB/s) with 1 file(s) remaining  
Completed 1.8 MiB/174.0 MiB (513.0 KiB/s) with 1 file(s) remaining  
Completed 2.0 MiB/174.0 MiB (576.9 KiB/s) with 1 file(s) remaining  
Completed 2.2 MiB/174.0 MiB (644.3 KiB/s) with 1 file(s) remaining  
Completed 2.5 MiB/174.0 MiB (703.5 KiB/s) with 1 file(s) remaining  
Completed 2.8 MiB/174.0 MiB (671.6 KiB/s) with 1 file(s) remaining  
Completed 3.0 MiB/174.0 MiB (728.8 KiB/s) with 1 file(s) remaining  
Completed 3.2 MiB/174.0 MiB (749.0 KiB/s) with 1 file(s) remaining  
Completed 3.5 MiB/174.0 MiB (805.8 KiB/s) with 1 file(s) remaining  
Completed 3.8 MiB/174.0 MiB (827.4 